# Digital Mapping for Humanists: A Geoparsing Workshop

Asko Nivala

University of Turku

## The aim of the workshop
This hands-on workshop introduces the basic principles of geoparsing – the process of identifying and interpreting place references in textual sources. Through practical exercises and group discussion, participants will learn how spatial information embedded in historical and literary texts can be extracted, visualised and analysed to support new kinds of research questions. No prior programming skills are required. By the end of the session, participants will have experimented with accessible tools to explore how texts can be transformed into maps, and how spatial approaches can enrich cultural and historical analysis.

## About this notebook
The workshop is based on this Jupyter Notebook. A Jupyter Notebook is a digital notebook that combines code, text and data in a single, interactive document. It allows researchers to write explanations, run small pieces of code (such as for analysing texts), and immediately see the results -- all in the same place. For humanists, it offers a gentle entry into digital research: you can explore language, visualise historical data or map places mentioned in texts, even with minimal programming experience.

This notebook will walk you through the geoparsing process and show some examples of Python code. Unfortunately, there is no ready-made software for geoparsing with a graphical user interface. You can execute each Python code cell by pressing the play button. Above the cell is documentation explaining what the code actually does.

## Installing Google Colab dependencies
If you want to run this notebook on Google Colab, you need to install the required libraries. This step is not necessary in Binder and might result in error.

In [1]:
!pip install -r https://raw.githubusercontent.com/askonivala/geodemo/main/requirements.txt

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.9 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of en-core-web-sm to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install -r https://raw.githubusercontent.com/askonivala/geodemo/main/requirements.txt (line 4) and spacy==3.8.7 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested spacy==3.8.7
    en-core-web-sm 3.7.1 depends on spacy<3.8.0 and >=3.7.2

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-depend

## Importing libraries and creating NLP objects
Then we need to import required Python libraries. A Python library is a collection of prewritten code that provides tools for specific tasks, such as analysing text or creating maps. To use a library in your code, you import it at the beginning of your notebook with a command like `import spacy` or `import folium`.

In [24]:
import spacy
from geopy.geocoders import Nominatim
import folium

Next, we will load the English language model into SpaCy. Please note that this model needs to be installed in console before the first run: `python -m spacy download en_core_web_sm`.

After that line, we will load Nominatim. This line creates a geolocator object using the Nominatim service, allowing you to search for place names, with "geo_tutorial" identifying your application.

In [25]:
nlp = spacy.load("en_core_web_sm")
geolocator = Nominatim(user_agent="geo_tutorial")

## NER (named-entity recognition)
The following code cell defines the text for which we will perform geoparsing analysis. You can place the text in the variable `text`. Please note that quotation marks must be closed for the line to work. In addition, the analysed text cannot contain quotation marks unless they are escaped appropriately.

In [26]:
text = "In the summer of 1802, Mary visited Geneva, passed through Milan, and rested in Florence."

These two lines use a language model to find place names in the text. First, the `nlp(text)` command tells the computer to analyse the text using natural language processing, in other words the English language model that we load on SpaCy. Then, the second line goes through the results and collects all the words or phrases that the model recognises as geographical places (like cities or countries). They are encoded with "GPE" (geopolitical entity) tag. Then we use Python's list comprehension to save them into a list called `places`.

In [27]:
doc = nlp(text)
places = [ent.text for ent in doc.ents if ent.label_ == "GPE"]

We can also check what kind of results were saved at this stage, although this is not required for the execution of the programme. We can see that document might entail also other entities than places, which we have selected based on "GPE" tag.

In [28]:
for entity in doc.ents:
    print(entity.text, entity.label_)
print(places)

the summer of 1802 DATE
Mary PERSON
Geneva GPE
Milan GPE
Florence GPE
['Geneva', 'Milan', 'Florence']


## Geolocating
Now we can try to find the coordinates for the place names using Nominatim. This block of code tries to find real-world map coordinates for each place name. For every place in the list, it asks an online map service (called a geolocator) to look it up. If the service finds a match, the place’s name along with its latitude and longitude (its exact position on Earth) are saved into a new list called `coords`. Finally, we print the contents of the variable to illustrate the results.

This step uses an external server, so it may take a long time or end with an error. That is why this implementation uses cache: in other words, it remembers which places have already been looked up, so if the same name appears again (like “Paris” in two different texts), it will not make a new request for the map service. This makes the code faster and more efficient — especially when analysing many texts.

In [34]:
coords = []
if not cache:
    cache = {}  # Dictionary to store already-looked-up places

for place in places:
    if place in cache:
        location = cache[place]
    else:
        location = geolocator.geocode(place)
        cache[place] = location

    if location:
        coords.append((place, location.latitude, location.longitude))
print(coords)

[('Geneva', 46.2017559, 6.1466014), ('Milan', 45.4641943, 9.1896346), ('Florence', 43.7697955, 11.2556404)]


## Mapping
This code creates an interactive map and adds a marker for each place with known coordinates. It starts by centering the map roughly over Central Europe. Obviously, if you are entering a text that is located to elsewhere, you need to modify these values or zoom out manually. Then it adds a marker at each latitude and longitude from the coords list. When you click on a marker, it shows the name of the place. The final line displays the map in the notebook.

In [23]:
map_ = folium.Map(location=[48, 10], zoom_start=4)
for name, lat, lon in coords:
    folium.Marker(location=[lat, lon], popup=name).add_to(map_)
map_